In [1]:
import sys
import requests as rq
from bs4 import BeautifulSoup as bs
from time import sleep
from time import time
from random import randint
from warnings import warn
import json
import pandas as pd

In [19]:
# d = {'piece': [], 'roundup': [], 'bulletin_url': [], 'bulletin_mps': []}
# df = pd.DataFrame(data=d)

In [85]:
# [len([j['href'] for j in i]) for i in df['bulletin_url']]

In [131]:
# df = dfcopy.copy()

In [217]:
# df_wi_bul = df[df['bulletin_url'].apply(lambda x: x[0] != 'E' )]
df_no_bul = df[df['bulletin_url'].apply(lambda x: x[0] == 'E' )]
df_no_bul

,piece,roundup,bulletin_url,bulletin_mps
3,"[\n, [Retail and recreation activity close to ...",Retail and recreation activity close to pre-co...,"[E, R, R, , :, :, , E, M, P, T, Y, , B, R, ...",placeholder
117,"[\n, [Ethnic minorities less likely to have ta...",Some ethnic groups are significantly less like...,"[E, R, R, , :, :, , E, M, P, T, Y, , B, R, ...",placeholder
120,"[\n, [Coronavirus (COVID-19) deaths rose in En...",Coronavirus (COVID-19) deaths rose in England ...,"[E, R, R, :, , U, R, L, , F, A, I, L, E, D]",placeholder
163,"[\n, [Previously infected people less likely t...",Previously infected people less likely to be v...,"[E, R, R, :, , U, R, L, , F, A, I, L, E, D]",placeholder
171,"[\n, [Estimates continue to show low rates of ...",Estimates continue to show low rates of corona...,"[E, R, R, , :, :, , E, M, P, T, Y, , B, R, ...",placeholder
...,...,...,...,...
550,"[\n, [UK labour market], [Our , [latest figure...",Our latest figures on the UK labour market hav...,"[E, R, R, :, , U, R, L, , F, A, I, L, E, D]",placeholder
551,"[\n, [Deaths involving COVID-19], [Up to 4 Sep...","Up to 4 September 2020, there were 52,376 deat...","[E, R, R, :, , U, R, L, , F, A, I, L, E, D]",placeholder
552,"[\n, [UK labour market], [Our , [latest figure...",Our latest figures on the UK labour market hav...,"[E, R, R, :, , U, R, L, , F, A, I, L, E, D]",placeholder
554,"[\n, [Mergers and acquisitions], [[Mergers and...",Mergers and acquisitions involving UK companie...,"[E, R, R, :, , U, R, L, , F, A, I, L, E, D]",placeholder


In [216]:
[i for i in df_wi_bul['bulletin_url']]

[['https://www.ons.gov.uk/peoplepopulationandcommunity/birthsdeathsandmarriages/deaths/bulletins/deathsregisteredweeklyinenglandandwalesprovisional/weekending15avril2022'],
 ['https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/bulletins/coronaviruscovid19infectionsurveypilot/22april2022'],
 ['https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19infectionsurveytechnicalarticlecumulativeincidenceofthenumberofpeoplewhohavetestedpositiveforcovid19uk/22april2022'],
 ['https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19infectionsurveytechnicalarticlecharacteristicsassociatedwiththirdvaccinationuptake/21april2022'],
 ['https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/healthandwellbeing/bulletins/coronavirusandselfisolationaftertestingpositiveinengland/17to26march2022'],
 ['https://www.ons.gov

In [201]:
df_multiurl = df[df['bulletin_url'].apply(lambda x: (len(x) > 1) & (x[0] != 'E') )]


In [213]:
df_multiurl.iloc[2]['piece']

<section>
<h3 id="perceptions">Distrust and perceived risks top reasons for vaccine refusal</h3><h4>16 March 2022</h4><p>“Side effects of the vaccine frightened me” </p><p>“I cannot do anything that may risk my fertility” </p><p>“If my body can fight it without a vaccine that’s good enough for me” </p><p>These are just some of the free-text responses collected as part of the Vaccine Opinions Study which provide insight into <a href="https://web.archive.org/web/20220323192308/https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/healthandwellbeing/bulletins/coronavirusandchangingattitudestowardsvaccinationengland/7to16september2021">why some people remain unvaccinated</a>. </p><p>Reasons for not vaccinating have been grouped into seven broad themes; </p>
<ul>
<li>feeling that the risks of a COVID-19 vaccine were too high or the benefits were too low</li>
<li>distrusting or feeling discontent towards vaccine stakeholders</li>
<li>lacking sufficient, trustworthy or favou

In [211]:
df_multiurl.iloc[2]['bulletin_url']

['https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/healthandwellbeing/bulletins/coronavirusandchangingattitudestowardsvaccinationengland/7to16september2021',
 'https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/healthandwellbeing/bulletins/coronavirusandchangingattitudestowardsvaccinationengland/7to16september2021',
 'https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/healthandwellbeing/articles/coronavirusandchangingperceptionstowardsvaccinationengland/7to16september2021']

In [212]:
df

,piece,roundup,bulletin_url,bulletin_mps
0,"[\n, [10.1% of registered deaths in England an...",10.1% of registered deaths in England and Wale...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
1,"[\n, [Infection rates decrease across the UK],...",Infection rates decrease across the UK\nThe pe...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
2,"[\n, [7 in 10 people in England have had COVID...",7 in 10 people in England have had COVID-19 \n...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
3,"[\n, [Retail and recreation activity close to ...",Retail and recreation activity close to pre-co...,"[E, R, R, , :, :, , E, M, P, T, Y, , B, R, ...",placeholder
4,"[\n, [People in deprived areas less likely to ...",People in deprived areas less likely to have h...,[https://www.ons.gov.uk/peoplepopulationandcom...,placeholder
...,...,...,...,...
551,"[\n, [Deaths involving COVID-19], [Up to 4 Sep...","Up to 4 September 2020, there were 52,376 deat...","[E, R, R, :, , U, R, L, , F, A, I, L, E, D]",placeholder
552,"[\n, [UK labour market], [Our , [latest figure...",Our latest figures on the UK labour market hav...,"[E, R, R, :, , U, R, L, , F, A, I, L, E, D]",placeholder
553,"[\n, [GDP, July 2020], [Monthly gross domestic...",Monthly gross domestic product (GDP) grew by 6...,[https://www.ons.gov.uk/chartimage?uri=/people...,placeholder
554,"[\n, [Mergers and acquisitions], [[Mergers and...",Mergers and acquisitions involving UK companie...,"[E, R, R, :, , U, R, L, , F, A, I, L, E, D]",placeholder


In [24]:
# ONS LATEST INSIGHTS APRIL Wayback machine archive urls
# url = 'http://web.archive.org/cdx/search/cdx?url=https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19/latestinsights&collapse=digest&from=20210601&to=20220501&output=json'
url = 'http://web.archive.org/cdx/search/cdx?url=ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/articles/coronaviruscovid19roundup/2020-03-26&collapse=digest&from=20200101&to=20211231&output=json'

urls = rq.get(url).text
parse_url = json.loads(urls) #parses the JSON from urls.

## Extracts timestamp and original columns from urls and compiles a url list.
url_list = []
for i in range(1,len(parse_url)):
    orig_url = parse_url[i][2]
    tstamp = parse_url[i][1]
    waylink = tstamp+'/'+orig_url
    url_list.append(waylink)
## Compiles final url pattern.
final_urls = []
for url in url_list:
    final_urls.append('https://web.archive.org/web/'+url)

In [8]:
def getpieces(url):
    # Open page for roundup
    req = rq.get(url).text
    # parse roundup html using beautifulsoup and store in soup
    soup = bs(req,'html.parser')

    # Find the section elements from the page
    sections = soup.find_all('section')
    # Filter out ones without h3 elements (headlines change between h3 and h2)
    pieces = []
    for section in sections:
        if section.find_all('h3') != []:
            pieces.append(section)

    pieces = [i for i in pieces if i.find('h3').text not in ["Latest COVID-19 headlines", "Explore by topic", "Also published recently"]]
    return pieces

In [10]:
# This is a function to format a single roundup entry 
def makeru(p):
        
    # Put in a list and remove line breaks
    p = [i for i in p if i not in ['\n']]
    p = [i for i in p if (i.text not in ['Analysis | Data', 'Download the data'])&('Embed this interactive' not in i.text)]
    
    p2 = []
    last = 'Not Notes'
    for n, i in enumerate(p):
        if (len(i.text) < 20) & (n < 3):
            pass
        else:
            if i.text == 'Notes':
                last = 'Notes'
            else:
                if last != 'Notes':
                    p2.append(i)
                last = 'Not Notes'
                
    
    # Remove all h4s and h5s which are the dates and chart titles, filter out elements that contain text suggesting its furniture
    p2 = [i for i in p2 if (i.name not in ['h2', 'h4', 'h5', 'summary', 'div']) and ('Embed code' not in i.text) and ('XLSX' not in i.text) and ('Explore the latest' not in i.text) and ('Read the latest' not in i.text) and ('Read the full bulletin' not in i.text) and ('Back to the top' not in i.text) and ('Download this chart' not in i.text)]
        
    # For each element in the section, if it is a list iteratre the list adding a dash to the listed text, if not a list then just get text from element
    p2 = [ [ " - " + i.text for i in q if i != '\n' ] if q.name == 'ul' else q.text for q in p2 ]

    string = ""
    
    for i in p2:
        if type(i) == list:
            for j in i:
                string = string + j + '\n'
        else:
            string = string + i + '\n'
        
    return string

# print(makeru(pieces[5]))

In [125]:
def getbullurl(piece):
    reldate = getdate(piece)
    
    loas = [i['href'] for i in piece.findAll('a') if ('articles/coronaviruscovid19latestinsights' not in i['href']) and ('ons.gov.uk' in i['href']) ]
    loas = ["http" + i.split("http")[-1]  for i in loas]
    
    links = []
    
    for e in loas:
        if e.split("/")[-1] == 'latest':
            
            req = rq.get(e).text
            soup = bs(req,'html.parser')
            prevrelurl = "https://www.ons.gov.uk" + [i for i in soup.find_all("a") if i.text =='View previous releases'][0]['href']
            
            linkseg = 'null'
            for i in range(1, 20):
                req = rq.get( prevrelurl + "?page=" + str(i) ).text
                soup = bs(req,'html.parser')
                
                if '404 - ' not in soup.find('title').text:
                    
                    
                    lis = soup.find("main").find("div", class_="print-links").find_all('li')
                    
                    if len([i.find("a") for i in lis if reldate in i.text]) > 0:
                        linkseg = [i.find("a") for i in lis if reldate in i.text][0]['href']
                        break
                        
            if linkseg == 'null':
                
                if 'Coronavirus and the social impacts' in soup.find('h1').text:
                    prevrelurl = "https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/healthandwellbeing/bulletins/coronavirusandthesocialimpactsongreatbritain/previousReleases"

                elif 'Economic activity and social change' in soup.find('h1').text:
                    prevrelurl = 'https://www.ons.gov.uk/economy/economicoutputandproductivity/output/bulletins/economicactivityandsocialchangeintheukrealtimeindicators/previousReleases'
                
                for i in range(1, 20):
                    req = rq.get( prevrelurl + "?page=" + str(i) ).text
                    soup = bs(req,'html.parser')

                    if '404 - ' not in soup.find('title').text:
                        lis = soup.find("main").find("div", class_="print-links").find_all('li')

                        if len([i.find("a") for i in lis if reldate in i.text]) > 0:
                            linkseg = [i.find("a") for i in lis if reldate in i.text][0]['href']
                            break
                            
            if linkseg == 'null':
                belletinurl = "ERR: NO PREVIOUS RELEASE FOR :: " + e + " DATE :: " + date
            else:
                bulletinurl = "https://www.ons.gov.uk" + linkseg
            
        elif '/releases/' not in e:
            bulletinurl = e
            
        else:    
            req = rq.get(e).text
            # parse roundup html using beautifulsoup and store in soup
            soup = bs(req,'html.parser')

            cols = soup.find_all('div', class_="col-wrap")

            pubs = [i for i in cols if 'Publications' in i.text]

            if len(pubs)>0:
                pub = pubs[0]
                pubind = cols.index(pub)
                linksect = cols[pubind+1]
                bulletinurl = "https://www.ons.gov.uk" + linksect.find("a")['href']

            else:
                bulletinurl = "ERR: NO PUBLICATION FOR :: " + e + " DATE :: " + date

        bulletinurl = bulletinurl.replace("https://ons", "https://www.ons")
        
        links.append(bulletinurl)
                
    return links
getbullurl(df['piece'][0])

['https://www.ons.gov.uk/peoplepopulationandcommunity/birthsdeathsandmarriages/deaths/bulletins/deathsregisteredweeklyinenglandandwalesprovisional/weekending15avril2022']

In [122]:
def getdate(piece):
    date = [i for i in piece if i != '\n']
    date = [i for i in date if len(i.text) < 18 ][0].text
    return "Released: " + date.rstrip()
    
getdate(df['piece'][0])

'Released: 26 April 2022'

In [51]:
def getbulletinurl(piece):

    p = [i for i in piece if i != '\n']
                
    try:
        p = [i for i in p if ('Read the bulletin' in i.text)|('Explore the latest insights' in i.text)|('Read the full' in i.text)|('Read the latest' in i.text)][0]
        if len(p.find_all('li'))>0:
            p = p.find_all('li')
            p = [i for i in p if ('latest insights' not in i.text) and ('latest indicators' not in i.text) and ("href=" in str(i))][0]
        p = p.find_all('a')

    except:
        try:
            p = [i for i in p if ('Read more' in i.text)|('Full bulletin' in i.text)][0]
            if len(p.find_all('li'))>0:
                p = p.find_all('li')
                p = [i for i in p if ('latest insights' not in i.text) and ('latest indicators' not in i.text) and ("href=" in str(i))][0]
            p = p.find_all('a')
        except:
            try:
                p = [i for i in p if ('Get the' in i.text)|('Article on' in i.text)|('Read' in i.text)|('Latest bulletin' in i.text)][0]
                if len(p.find_all('li'))>0:
                    p = p.find_all('li')
                    p = [i for i in p if ('latest insights' not in i.text) and ('latest indicators' not in i.text) and ("href=" in str(i))][0]
                p = p.find_all('a')
            except:
                try:
                    p = [i for i in p if 'Latest real-time indicators' in i.text][0]
                    if len(p.find_all('li'))>0:
                        p = p.find_all('li')
                        p = [i for i in p if ('latest insights' not in i.text) and ('latest indicators' not in i.text) and ("href=" in str(i))][0]
                    p = p.find_all('a')
                except:
                    try:
                        p = [i for i in p if ('Explore the latest indicators' in i.text)|('Analysis' == i.text)][0]
                        if len(p.find_all('li'))>0:
                            p = p.find_all('li')
                            p = [i for i in p if ('latest insights' not in i.text) and ('latest indicators' not in i.text) and ("href=" in str(i))][0]
                        p = p.find_all('a')
                    except:
                        return "ERR: NO LINK"

    p = p[0]['href']
    p = "http" + p.split("http")[-1]
    p = p.replace("https:/www", "https://www")
        
    date = piece.find('p').text
    reldate = "Released: " + date
#     date = piece.find('h4').text
    
                    
    if p.split("/")[-1] == 'latest':
                
        req = rq.get(p).text
        soup = bs(req,'html.parser')

        prevrelurl = "https://www.ons.gov.uk" + [i for i in soup.find_all("a") if i.text =='View previous releases'][0]['href']
                
#         if piece.find('h3').text == 'UK online job adverts in catering and hospitality rise above pre-pandemic levels':
#             prevrelurl = "https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/healthandwellbeing/bulletins/coronavirusandthesocialimpactsongreatbritain/previousReleases"
         
#         if piece.find('h3').text == 'UK seated diner reservations increase by 13 percentage points':
#             prevrelurl = 'https://www.ons.gov.uk/economy/economicoutputandproductivity/output/bulletins/economicactivityandsocialchangeintheukrealtimeindicators/previousReleases'
        
        req = rq.get(prevrelurl).text
        soup = bs(req,'html.parser')

        lis = soup.find("main").find("div", class_="print-links").find_all('li')
                
        try:
            linkseg = [i.find("a") for i in lis if date in i.text][0]['href']
        except:
            
            for i in range(1, 20):
                req = rq.get( prevrelurl + "?page=" + str(i) ).text
                soup = bs(req,'html.parser')
                
                if '404 - ' not in soup.find('title').text:
                    lis = soup.find("main").find("div", class_="print-links").find_all('li')

                    if len([i.find("a") for i in lis if reldate.rstrip() in i.text]) > 0:
                        linkseg = [i.find("a") for i in lis if reldate.rstrip() in i.text][0]['href']
                        break
                        
            if 'Coronavirus and the social impacts' in soup.find('h1').text:
                prevrelurl = "https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/healthandwellbeing/bulletins/coronavirusandthesocialimpactsongreatbritain/previousReleases"

            elif 'Economic activity and social change' in soup.find('h1').text:
                prevrelurl = 'https://www.ons.gov.uk/economy/economicoutputandproductivity/output/bulletins/economicactivityandsocialchangeintheukrealtimeindicators/previousReleases'

            for i in range(1, 20):
                req = rq.get( prevrelurl + "?page=" + str(i) ).text
                soup = bs(req,'html.parser')
                
                if '404 - ' not in soup.find('title').text:
                    lis = soup.find("main").find("div", class_="print-links").find_all('li')

                    if len([i.find("a") for i in lis if reldate.rstrip() in i.text]) > 0:
                        linkseg = [i.find("a") for i in lis if reldate.rstrip() in i.text][0]['href']
                        break

        bulletinurl = "https://www.ons.gov.uk" + linkseg
        
    elif '/releases/' not in p:
        bulletinurl = p
    else:    
        req = rq.get(p).text
        # parse roundup html using beautifulsoup and store in soup
        soup = bs(req,'html.parser')
        
        cols = soup.find_all('div', class_="col-wrap")
        
        pubs = [i for i in cols if 'Publications' in i.text]
        
        if len(pubs)>0:
            pub = pubs[0]
            pubind = cols.index(pub)
            linksect = cols[pubind+1]
            bulletinurl = "https://www.ons.gov.uk" + linksect.find("a")['href']
            
        else:
            bulletinurl = "ERR: NO PUBLICATION"
            
    bulletinurl = bulletinurl.replace("https://ons", "https://www.ons")
    
    return bulletinurl

# getbulletinurl(pieces[4])

In [16]:
def addpiecestodf(pieces, df):
    for j, piece in enumerate(pieces):
        if makeru(piece) not in [el for el in df['roundup']]:
            df = df.append({'piece': piece, 'roundup': makeru(piece), 'bulletin_url': 'placeholder', 'bulletin_mps': 'placeholder'}, ignore_index=True)

    return df


In [ ]:
for i in range(1, len(final_urls)+1):
    print('i', i)
    print(final_urls[-i])
    pieces = getpieces(final_urls[-i])
    df = addpiecestodf(pieces, df)
df

In [ ]:
def addbulurlodf(pieces, df):
    for j, piece in enumerate(pieces):
        if makeru(piece) not in [el for el in df['roundup']]:
            df = df.append({'piece': piece, 'roundup': makeru(piece), 'bulletin_url': 'placeholder', 'bulletin_mps': 'placeholder'}, ignore_index=True)

    return df


In [133]:
for i in df.index:
    if df.iloc[i]['bulletin_url'] == 'placeholder':
        try:
            if getbullurl(df.iloc[i]['piece']) == []:
                df.iloc[i]['bulletin_url'] = 'ERR: NO URL FOUND'
            else:
                df.iloc[i]['bulletin_url'] = getbullurl(df.iloc[i]['piece'])
        except:
            df.iloc[i]['bulletin_url'] = "ERR: URL FAILED"
            

In [182]:
for i in df.index:
    notin = ['/methodology/', '.xlsx']
    for j in notin:
        df['bulletin_url'][i] = [i for i in df['bulletin_url'][i] if j not in i]
    if df['bulletin_url'][i] == []:
        df['bulletin_url'][i] = 'ERR :: EMPTY BRACKETS'

In [226]:
def getmps(bulletinurl):
    
    # Open page for roundup
    req = rq.get(bulletinurl).text
    # parse roundup html using beautifulsoup and store in soup
    soup = bs(req,'html.parser')
        
    sects = [sect for sect in soup.find_all('section') if sect.find_all('h2') != []]    
    sect = [sect for sect in sects if 'Main points' in sect.find('h2').text]
    
    if len(sect)>0:
                
        sect = sect[0]

        lis = sect.find('ul')
        if lis != None:
            sibling = lis.next_sibling

            while sibling:
                next_sibling = sibling.next_sibling
                sibling.extract()
                sibling = next_sibling
            
        ps = []
        for i in sect:
            if i.name == 'p':
                ps.append(i.text)

            elif i.name == 'ul':
                for j in i.find_all('li'):
                    ps.append(j.text)            
            
    
    else:
        return "NO MAIN POINTS"
        
    mps = [i for i in ps if ('Source: Office for National Statistics' not in i) and ('Find out more' not in i) and ('make the website work as well as possible' not in i) and ('Embed this interactive' not in i) and ('xlsx' not in i) and ('cookies' not in i)]
    mainpoints = ""
    for i in mps: 
        mainpoints = mainpoints + i + '\n' 
    
    return mainpoints

print(getmps(df_wi_bul['bulletin_url'][0][0]))

In the week ending 15 April 2022 (Week 15), 9,919 deaths were registered in England and Wales; of these deaths, 1,003 mentioned "novel coronavirus (COVID-19)", accounting for 10.1% of all deaths.
This is an increase in COVID-19 deaths compared with the week ending 1 April 2022 (Week 14) when the number of all-cause deaths registered was 10,687; of these, COVID-19 accounted for 960 deaths (9.0%).
Deaths registered in Week 15 are affected by the Good Friday bank holiday. The five-year average for Week 15 includes a bank holiday in one of the five years.
Of the 1,003 deaths involving COVID-19, 64.2% (644 deaths) had this recorded as the underlying cause of death in Week 15, compared with 64.3% in Week 14.
In Week 15 in England and Wales, the number of deaths was above the five-year average in private homes (6.9% above, 181 excess deaths) but below the five-year average in care homes (13.2% below, 292 fewer deaths), hospitals (9.3% below, 462 fewer deaths), and other settings (12.3% below,

In [280]:
[i for i in df_wi_bul['bulletin_mps']].index( 'In England, an estimated 7 in 10 adults or 69.3% of the adult population (95% credible interval: 65.6% to 73.9%) would have tested positive for antibodies against the coronavirus – SARS-CoV-2 – on a blood test in the week beginning 19 April 2021, suggesting they had the infection in the past or have been vaccinated.\nIn Wales, an estimated 6 in 10 adults, or 63.2% of the adult population (95% credible interval: 58.8% to 68.3%) would have tested positive for antibodies against SARS-CoV-2 on a blood test in the week beginning 19 April 2021, suggesting they had the infection in the past or have been vaccinated.\nIn Northern Ireland, an estimated 6 in 10 adults, or 63.5% of the adult population (95% credible interval: 57.6% to 71.2%) would have tested positive for antibodies against SARS-CoV-2 on a blood test in the week beginning 19 April 2021, suggesting they had the infection in the past or have been vaccinated.\nIn Scotland, an estimated 6 in 10 adults, or 59.2% of the adult population (95% credible interval: 54.8% to 64.4%) would have tested positive for antibodies against SARS-CoV-2 on a blood test in the week beginning 19 April 2021, suggesting they had the infection in the past or have been vaccinated.\nAcross all four countries of the UK, there is a clear pattern between vaccination and testing positive for COVID-19 antibodies but the detection of antibodies alone is not a precise measure of the immunity protection given by vaccination.\n',
)

352

In [285]:
df_wi_bul

,piece,roundup,bulletin_url,bulletin_mps
0,"[\n, [10.1% of registered deaths in England an...",10.1% of registered deaths in England and Wale...,[https://www.ons.gov.uk/peoplepopulationandcom...,"In the week ending 15 April 2022 (Week 15), 9,..."
1,"[\n, [Infection rates decrease across the UK],...",Infection rates decrease across the UK\nThe pe...,[https://www.ons.gov.uk/peoplepopulationandcom...,"In England, the percentage of people testing p..."
2,"[\n, [7 in 10 people in England have had COVID...",7 in 10 people in England have had COVID-19 \n...,[https://www.ons.gov.uk/peoplepopulationandcom...,"In England, an estimated 38.5 million people h..."
4,"[\n, [People in deprived areas less likely to ...",People in deprived areas less likely to have h...,[https://www.ons.gov.uk/peoplepopulationandcom...,"Across all ages, of those eligible, the follow..."
5,"[\n, [Fall in self-isolation compliance after ...",Fall in self-isolation compliance after testin...,[https://www.ons.gov.uk/peoplepopulationandcom...,Respondents in this survey tested positive for...
...,...,...,...,...
545,"[\n, [Retail sales up 4.0% compared with pre-p...",Retail sales volumes (the amount of goods boug...,[https://www.ons.gov.uk/chartimage?uri=/people...,placeholder
546,"[\n, [Monthly mortality analysis August 2020],...","There were 34,750 deaths registered in England...",[https://www.ons.gov.uk/peoplepopulationandcom...,"In August 2020, there were 34,750 deaths regis..."
547,"[\n, [Levels of socialising in the UK have dec...","Levels of socialising, eating out and travel i...",[https://www.ons.gov.uk/chartimage?uri=/people...,placeholder
549,"[\n, [Average house prices continued to rise i...",The UK’s average house prices increased by 3.4...,[https://www.ons.gov.uk/economy/inflationandpr...,UK average house prices increased by 3.4% over...


In [286]:
df_wi_mps = df_wi_bul[df_wi_bul['bulletin_mps'].apply(lambda x: x != 'placeholder' )]

In [287]:
df_wi_mps

,piece,roundup,bulletin_url,bulletin_mps
0,"[\n, [10.1% of registered deaths in England an...",10.1% of registered deaths in England and Wale...,[https://www.ons.gov.uk/peoplepopulationandcom...,"In the week ending 15 April 2022 (Week 15), 9,..."
1,"[\n, [Infection rates decrease across the UK],...",Infection rates decrease across the UK\nThe pe...,[https://www.ons.gov.uk/peoplepopulationandcom...,"In England, the percentage of people testing p..."
2,"[\n, [7 in 10 people in England have had COVID...",7 in 10 people in England have had COVID-19 \n...,[https://www.ons.gov.uk/peoplepopulationandcom...,"In England, an estimated 38.5 million people h..."
4,"[\n, [People in deprived areas less likely to ...",People in deprived areas less likely to have h...,[https://www.ons.gov.uk/peoplepopulationandcom...,"Across all ages, of those eligible, the follow..."
5,"[\n, [Fall in self-isolation compliance after ...",Fall in self-isolation compliance after testin...,[https://www.ons.gov.uk/peoplepopulationandcom...,Respondents in this survey tested positive for...
...,...,...,...,...
532,"[\n, [Average house prices continued to rise i...",The UK’s average house prices increased by 2.3...,[https://www.ons.gov.uk/economy/inflationandpr...,UK average house prices increased by 2.3% over...
541,"[\n, [Deaths involving COVID-19 by occupation]...",Among men working in health care and social ca...,[https://www.ons.gov.uk/peoplepopulationandcom...,Building on previous releases that looked at r...
544,"[\n, [Deaths involving COVID-19 by occupation]...",Among men working in health care and social ca...,[https://www.ons.gov.uk/peoplepopulationandcom...,Building on previous releases that looked at r...
546,"[\n, [Monthly mortality analysis August 2020],...","There were 34,750 deaths registered in England...",[https://www.ons.gov.uk/peoplepopulationandcom...,"In August 2020, there were 34,750 deaths regis..."


In [300]:
[i for i in df_wi_mps['bulletin_mps']]

['In the week ending 15 April 2022 (Week 15), 9,919 deaths were registered in England and Wales; of these deaths, 1,003 mentioned "novel coronavirus (COVID-19)", accounting for 10.1% of all deaths.\nThis is an increase in COVID-19 deaths compared with the week ending 1 April 2022 (Week 14) when the number of all-cause deaths registered was 10,687; of these, COVID-19 accounted for 960 deaths (9.0%).\nDeaths registered in Week 15 are affected by the Good Friday bank holiday. The five-year average for Week 15 includes a bank holiday in one of the five years.\nOf the 1,003 deaths involving COVID-19, 64.2% (644 deaths) had this recorded as the underlying cause of death in Week 15, compared with 64.3% in Week 14.\nIn Week 15 in England and Wales, the number of deaths was above the five-year average in private homes (6.9% above, 181 excess deaths) but below the five-year average in care homes (13.2% below, 292 fewer deaths), hospitals (9.3% below, 462 fewer deaths), and other settings (12.3% 

In [283]:
for k, i in enumerate(df_wi_bul.index):
    if k > 352:
        for j in range(1, len(df_wi_bul['bulletin_url'][i]) + 1 ):
            url = df_wi_bul['bulletin_url'][i][-j]
            url = url.replace("http:/www", "https://www")
            mps = getmps(url)
            if mps != "NO MAIN POINTS":
                df_wi_bul['bulletin_mps'][i] = getmps(url)
                break

/Users/theojolliffe/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


ok


In [290]:
import random

In [299]:
roundupsum_train = {}
roundupsum_train['data'] = []

roundupsum_val = {}
roundupsum_val['data'] = []

In [296]:
for i in df_wi_mps.index:
    if random.random() < 0.05:
        roundupsum_val['data'].append({
            'document': df.iloc[i]['bulletin_mps'],
            'summary': df.iloc[i]['roundup'],
            'id': i
        })
    else:
        roundupsum_train['data'].append({
            'document': df.iloc[i]['bulletin_mps'],
            'summary': df.iloc[i]['roundup'],
            'id': i
        })

In [297]:
with open('roundupsum_train.json', 'w') as outfile:
    json.dump(roundupsum_train, outfile)
    
with open('roundupsum_val.json', 'w') as outfile:
    json.dump(roundupsum_val, outfile)

In [298]:
! git add .
! git commit -m "first commit"
! git push -u origin main

[main 92c3fd4] first commit
 3 files changed, 79 insertions(+), 8 deletions(-)
 rewrite roundupsum_train.json (85%)
 rewrite roundupsum_val.json (99%)
Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 16 threads
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 15.36 KiB | 1.40 MiB/s, done.
Total 5 (delta 4), reused 0 (delta 0)
remote: Resolving deltas: 100% (4/4), completed with 3 local objects.
To https://github.com/theojolliffe/roundupdata.git
   bb0fe42..92c3fd4  main -> main
Branch 'main' set up to track remote branch 'main' from 'origin'.
